# ASL Classification

#### Problem Statement:

Detecting and Classifying ASL Alphabets (A-Z).

#### Import Libraries:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import split_folders
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dropout,Dense,Flatten
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.applications import MobileNet
from PIL import ImageFile
from keras.preprocessing import image
from tensorflow.keras.applications import Xception
from tensorflow.keras.applications.xception import preprocess_input as mp
from warnings import filterwarnings
filterwarnings('ignore')

In [ ]:
# Splitting the dataset folder to Train and Validation
split_folders.ratio("Alphabets",output="Alphabets_split",seed=1337,ratio=(0.8, 0.2))

#### Data Augmentation :

In [ ]:
train_datagen=ImageDataGenerator(rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

train_data=train_datagen.flow_from_directory("Alphabets_split/train",target_size=(32, 32),class_mode='categorical',
    batch_size=8)

In [ ]:
val_datagen=ImageDataGenerator(rescale=1./255)

val_data=val_datagen.flow_from_directory("Alphabets_split/val",target_size=(32, 32),class_mode='categorical',
    batch_size=8)

#### VGG19 model :

In [ ]:
#Initialising the VGG19 model
model=VGG19()

In [ ]:
# Checking the entire structure of VGG19
model.summary()

In [ ]:
# Excluding the Convolution layers of the VGG19 model and setting the input shape same as that of target size during Data Augmentation
model=VGG19(input_shape=(32,32,3),include_top=False)

In [ ]:
# Train the weights of only the dense layers 
for layers in model.layers:
    if('dense' not in layers.name):
        layers.trainable=False
    if('dense' in layers.name):
        layers.trainable=True

In [ ]:
#Adding the new dense layers to the VGG19 model
transfer_model=Sequential([model,
                          Flatten(),
                          Dense(32,activation='relu'),
                          Dense(26,activation='softmax')])
transfer_model.summary()

In [ ]:
#Compiling the model using adam optimizer, loss as 'categorical_crossentropy' and metrics as accuracy
transfer_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:

ImageFile.LOAD_TRUNCATED_IMAGES = True
my_model=transfer_model.fit_generator(train_data,steps_per_epoch=518 //8,epochs=100,validation_data=val_data,validation_steps=130  //8)

In [ ]:
transfer_model.save("sign_detection.h5")

In [ ]:
# Comparing performance metrics of the model
plt.figure(figsize=(15,5))
plt.subplot(121)
plt.plot(my_model.history['accuracy'],label='Train')
plt.plot(my_model.history['val_accuracy'],label='Test')
plt.title("Accuracy")
plt.legend()

plt.subplot(122)
plt.plot(my_model.history['loss'],label='Train')
plt.plot(my_model.history['val_loss'],label='Test')
plt.title("Loss")
plt.legend()
plt.show()

#### Making a single Prediction:

In [ ]:
img1=cv2.imread("Alphabets_split/val/A/A626.jpg")
img1=cv2.cvtColor(img1,cv2.COLOR_BGR2RGB)
test_image = image.load_img('Alphabets_split/val/A/A626.jpg', target_size = (32, 32))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = transfer_model.predict(test_image/255.)
prediction=list(train_data.class_indices.keys())[np.argmax(result)]

plt.figure(figsize=(15,5))
plt.subplot(121)
plt.imshow(img1)
plt.title("Actual: A",fontsize=20)

plt.subplot(122)
plt.imshow(img1)
plt.title("Predicted: {}".format(prediction),fontsize=20)
plt.show()

In [ ]:
img1=cv2.imread("Alphabets_split/val/B/B691.jpg")
img1=cv2.cvtColor(img1,cv2.COLOR_BGR2RGB)
test_image = image.load_img('Alphabets_split/val/B/B691.jpg', target_size = (32, 32))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = transfer_model.predict(test_image/255.)
prediction=list(train_data.class_indices.keys())[np.argmax(result)]

plt.figure(figsize=(15,5))
plt.subplot(121)
plt.imshow(img1)
plt.title("Actual: B",fontsize=20)

plt.subplot(122)
plt.imshow(img1)
plt.title("Predicted: {}".format(prediction),fontsize=20)
plt.show()

In [ ]:
img1=cv2.imread("Alphabets_split/val/I/I578.jpg")
img1=cv2.cvtColor(img1,cv2.COLOR_BGR2RGB)
test_image = image.load_img('Alphabets_split/val/I/I578.jpg', target_size = (32, 32))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = transfer_model.predict(test_image/255.)
prediction=list(train_data.class_indices.keys())[np.argmax(result)]

plt.figure(figsize=(15,5))
plt.subplot(121)
plt.imshow(img1)
plt.title("Actual: I",fontsize=20)

plt.subplot(122)
plt.imshow(img1)
plt.title("Predicted: {}".format(prediction),fontsize=20)
plt.show()

In [ ]:
img1=cv2.imread("Alphabets_split/val/K/K530.jpg")
img1=cv2.cvtColor(img1,cv2.COLOR_BGR2RGB)
test_image = image.load_img('Alphabets_split/val/K/K530.jpg', target_size = (32, 32))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = transfer_model.predict(test_image/255.)
prediction=list(train_data.class_indices.keys())[np.argmax(result)]

plt.figure(figsize=(15,5))
plt.subplot(121)
plt.imshow(img1)
plt.title("Actual: K",fontsize=20)

plt.subplot(122)
plt.imshow(img1)
plt.title("Predicted: {}".format(prediction),fontsize=20)
plt.show()

#### Xception:

In [ ]:
train_datagen=ImageDataGenerator(rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

train_data=train_datagen.flow_from_directory("Alphabets_split/train",target_size=(128,128),class_mode='categorical',
    batch_size=8)

In [ ]:
val_datagen=ImageDataGenerator(rescale=1./255)

val_data=val_datagen.flow_from_directory("Alphabets_split/val",target_size=(128,128),class_mode='categorical',
    batch_size=8)

In [ ]:
model=Xception()

In [ ]:
model.summary()

In [ ]:
# Excluding the Convolution layers of the Xception model and setting the input shape same as that of target size during Data Augmentation
model=Xception(input_shape=(128,128,3),include_top=False)

In [ ]:
# Train the weights of only the dense layers 
for layers in model.layers:
    if('dense' not in layers.name):
        layers.trainable=False
    if('dense' in layers.name):
        layers.trainable=True

In [ ]:
#Adding the new dense layers to the Xception model
transfer_model=Sequential([model,
                          Flatten(),
                          Dense(32,activation='relu'),
                          Dense(26,activation='softmax')])
transfer_model.summary()

In [ ]:
#Compiling the model using adam optimizer, loss as 'categorical_crossentropy' and metrics as accuracy
transfer_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
ImageFile.LOAD_TRUNCATED_IMAGES = True
my_model=transfer_model.fit_generator(train_data,steps_per_epoch=518 //8,epochs=100,validation_data=val_data,validation_steps=130  //8)

In [ ]:
# Comparing performance metrics of the model
plt.figure(figsize=(15,5))
plt.subplot(121)
plt.plot(my_model.history['accuracy'],label='Train')
plt.plot(my_model.history['val_accuracy'],label='Test')
plt.title("Accuracy")
plt.legend()

plt.subplot(122)
plt.plot(my_model.history['loss'],label='Train')
plt.plot(my_model.history['val_loss'],label='Test')
plt.title("Loss")
plt.legend()
plt.show()

#### Make Single Predictions:

In [ ]:
img1=cv2.imread("Alphabets_split/val/A/A626.jpg")
img1=cv2.cvtColor(img1,cv2.COLOR_BGR2RGB)
test_image = image.load_img('Alphabets_split/val/A/A626.jpg', target_size = (128, 128))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = transfer_model.predict(test_image/255.)
prediction=list(train_data.class_indices.keys())[np.argmax(result)]

plt.figure(figsize=(15,5))
plt.subplot(121)
plt.imshow(img1)
plt.title("Actual: A",fontsize=20)

plt.subplot(122)
plt.imshow(img1)
plt.title("Predicted: {}".format(prediction),fontsize=20)
plt.show()

In [ ]:
img1=cv2.imread("Alphabets_split/val/B/B691.jpg")
img1=cv2.cvtColor(img1,cv2.COLOR_BGR2RGB)
test_image = image.load_img('Alphabets_split/val/B/B691.jpg', target_size = (128, 128))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = transfer_model.predict(test_image/255.)
prediction=list(train_data.class_indices.keys())[np.argmax(result)]

plt.figure(figsize=(15,5))
plt.subplot(121)
plt.imshow(img1)
plt.title("Actual: B",fontsize=20)

plt.subplot(122)
plt.imshow(img1)
plt.title("Predicted: {}".format(prediction),fontsize=20)
plt.show()

In [ ]:
img1=cv2.imread("Alphabets_split/val/I/I578.jpg")
img1=cv2.cvtColor(img1,cv2.COLOR_BGR2RGB)
test_image = image.load_img('Alphabets_split/val/I/I578.jpg', target_size = (128, 128))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = transfer_model.predict(test_image/255.)
prediction=list(train_data.class_indices.keys())[np.argmax(result)]

plt.figure(figsize=(15,5))
plt.subplot(121)
plt.imshow(img1)
plt.title("Actual: I",fontsize=20)

plt.subplot(122)
plt.imshow(img1)
plt.title("Predicted: {}".format(prediction),fontsize=20)
plt.show()

In [ ]:
img1=cv2.imread("Alphabets_split/val/K/K530.jpg")
img1=cv2.cvtColor(img1,cv2.COLOR_BGR2RGB)
test_image = image.load_img('Alphabets_split/val/K/K530.jpg', target_size = (128, 128))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = transfer_model.predict(test_image/255.)
prediction=list(train_data.class_indices.keys())[np.argmax(result)]

plt.figure(figsize=(15,5))
plt.subplot(121)
plt.imshow(img1)
plt.title("Actual: K",fontsize=20)

plt.subplot(122)
plt.imshow(img1)
plt.title("Predicted: {}".format(prediction),fontsize=20)
plt.show()

#### Mobile Net:

In [ ]:
train_datagen=ImageDataGenerator(rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

train_data=train_datagen.flow_from_directory("Alphabets_split/train",target_size=(32, 32),class_mode='categorical',
    batch_size=8)

In [ ]:
val_datagen=ImageDataGenerator(rescale=1./255)

val_data=val_datagen.flow_from_directory("Alphabets_split/val",target_size=(32, 32),class_mode='categorical',
    batch_size=8)

In [ ]:
model=MobileNet()

In [ ]:
model.summary()

In [ ]:
# Excluding the Convolution layers of the MobileNet model and setting the input shape same as that of target size during Data Augmentation
model=MobileNet(input_shape=(32,32,3),include_top=False)

In [ ]:
# Train the weights of only the dense layers 
for layers in model.layers:
    if('dense' not in layers.name):
        layers.trainable=False
    if('dense' in layers.name):
        layers.trainable=True

In [ ]:
#Adding the new dense layers to the MobileNet model
transfer_model=Sequential([model,
                          Flatten(),
                          Dense(32,activation='relu'),
                          Dense(26,activation='softmax')])
transfer_model.summary()

In [ ]:
#Compiling the model using adam optimizer, loss as 'categorical_crossentropy' and metrics as accuracy
transfer_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
ImageFile.LOAD_TRUNCATED_IMAGES = True
my_model=transfer_model.fit_generator(train_data,steps_per_epoch=518 //8,epochs=100,validation_data=val_data,validation_steps=130  //8)

In [ ]:
# Comparing performance metrics of the model
plt.figure(figsize=(15,5))
plt.subplot(121)
plt.plot(my_model.history['accuracy'],label='Train')
plt.plot(my_model.history['val_accuracy'],label='Test')
plt.title("Accuracy")
plt.legend()

plt.subplot(122)
plt.plot(my_model.history['loss'],label='Train')
plt.plot(my_model.history['val_loss'],label='Test')
plt.title("Loss")
plt.legend()
plt.show()

#### Make Single Predictions:

In [ ]:
img1=cv2.imread("Alphabets_split/val/A/A626.jpg")
img1=cv2.cvtColor(img1,cv2.COLOR_BGR2RGB)
test_image = image.load_img('Alphabets_split/val/A/A626.jpg', target_size = (32, 32))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = transfer_model.predict(test_image/255.)
prediction=list(train_data.class_indices.keys())[np.argmax(result)]

plt.figure(figsize=(15,5))
plt.subplot(121)
plt.imshow(img1)
plt.title("Actual: A",fontsize=20)

plt.subplot(122)
plt.imshow(img1)
plt.title("Predicted: {}".format(prediction),fontsize=20)
plt.show()

In [ ]:
img1=cv2.imread("Alphabets_split/val/B/B691.jpg")
img1=cv2.cvtColor(img1,cv2.COLOR_BGR2RGB)
test_image = image.load_img('Alphabets_split/val/B/B691.jpg', target_size = (32, 32))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = transfer_model.predict(test_image/255.)
prediction=list(train_data.class_indices.keys())[np.argmax(result)]

plt.figure(figsize=(15,5))
plt.subplot(121)
plt.imshow(img1)
plt.title("Actual: B",fontsize=20)

plt.subplot(122)
plt.imshow(img1)
plt.title("Predicted: {}".format(prediction),fontsize=20)
plt.show()

In [ ]:
img1=cv2.imread("Alphabets_split/val/I/I578.jpg")
img1=cv2.cvtColor(img1,cv2.COLOR_BGR2RGB)
test_image = image.load_img('Alphabets_split/val/I/I578.jpg', target_size = (32, 32))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = transfer_model.predict(test_image/255.)
prediction=list(train_data.class_indices.keys())[np.argmax(result)]

plt.figure(figsize=(15,5))
plt.subplot(121)
plt.imshow(img1)
plt.title("Actual: I",fontsize=20)

plt.subplot(122)
plt.imshow(img1)
plt.title("Predicted: {}".format(prediction),fontsize=20)
plt.show()

In [ ]:
img1=cv2.imread("Alphabets_split/val/K/K530.jpg")
img1=cv2.cvtColor(img1,cv2.COLOR_BGR2RGB)
test_image = image.load_img('Alphabets_split/val/K/K530.jpg', target_size = (32, 32))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = transfer_model.predict(test_image/255.)
prediction=list(train_data.class_indices.keys())[np.argmax(result)]

plt.figure(figsize=(15,5))
plt.subplot(121)
plt.imshow(img1)
plt.title("Actual: K",fontsize=20)

plt.subplot(122)
plt.imshow(img1)
plt.title("Predicted: {}".format(prediction),fontsize=20)
plt.show()

In [ ]:
from PIL import Image
from keras.models import load_model
import cv2

import numpy as np

In [ ]:
model=load_model("p_weights.h5")

In [ ]:
face_classifier = cv2.CascadeClassifier('D:/MTech-DSML/DATA/haarcascade_frontalface_alt2.xml')

In [ ]:
def feature_extractor(img):
    gesture=face_classifier.detectMultiScale(img,2.3,2)
    
    if gesture is ():
        return None
    
    for (x,y,w,h) in gesture:
        x=x-10
        y=y-10
        cropped=img[y:y+h+150,x:x+w+150]
        
    return cropped

In [ ]:
video_capture = cv2.VideoCapture(0)
while True:
    _,frame = video_capture.read()
    
    gest=feature_extractor(frame)
    if type(gest) is np.ndarray:
        gest = cv2.resize(gest, (32, 32))
        im = Image.fromarray(gest, 'RGB')
        
        img_array = np.array(im)
        img_array = np.expand_dims(img_array, axis=0)
        pred = model.predict(img_array)
        print(pred)
        
        name="None matching"
        
        if(pred[0][0]>0.5):
            name='A'
        elif (pred[0][1]>0.5) :
            name='B'
        cv2.putText(frame,name, (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
    else:
        cv2.putText(frame,"", (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
    cv2.imshow('Video', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
video_capture.release()
cv2.destroyAllWindows()